In [15]:
import pandas as pd
from vayu_gnn.dbx.dbx_config import dbx_helper, DropboxHelper

In [23]:
node_pickle = dbx_helper.read_pickle(dbx_helper.clean_input_path, 'node_locations/Patna', 'nodes.pickle') 
df = pd.DataFrame.from_dict(node_pickle, orient='index')
df = df.reset_index().rename(columns={'index': 'node_id'})
df = df.rename(columns={'long': 'lon'})
df = df[df['node_id'].str.startswith('T')]

df.to_csv('/Users/emmettsexton/Dropbox/Mac (2)/Desktop/EconAI/India air pollution hackathon/demo_nodes.csv', index=False)

In [19]:
predictions_in = dbx_helper.read_parquet(dbx_helper.output_path, 'Patna', 'predictions.parquet')
predictions = predictions_in.reset_index().copy()
predictions = predictions.melt(id_vars=['date', 'hour', 'horizon', 'node_id'], var_name='pollutant', value_name='value')
predictions['pollutant'] = predictions['pollutant'].str.upper()
predictions['pollutant'] = predictions['pollutant'].str.replace('PM_25', 'PM2.5')
predictions['pollutant'] = predictions['pollutant'].str.replace('PM_10', 'PM10')

File 'predictions.parquet' successfully downloaded and loaded into a DataFrame.


In [22]:
df

,node,lat,lon
0,TARA012,25.599409,85.113762
1,TARA013,25.606958,85.165016
2,TARA016,25.578127,85.056877
3,TARA017,25.622450,85.091530
4,TARA018,25.582579,85.097511
5,TARA019,25.569492,85.179970
6,TARA021,25.610352,85.132950
7,TARA022,25.619673,85.147415
8,TARA025,25.614792,85.099220
9,TARA026,25.636068,85.104805


In [20]:
predictions.to_csv('/Users/emmettsexton/Dropbox/Mac (2)/Desktop/EconAI/India air pollution hackathon/predictions.csv', index=False)

In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import folium
from streamlit_folium import st_folium
from branca.colormap import StepColormap

# Nodes around central Patna
np.random.seed(42)
n_nodes = 30
center_lat, center_lon = 25.5941, 85.1376
nodes = pd.DataFrame({
    'node_id': [f'Node {i+1}' for i in range(n_nodes)],
    'lat': center_lat + np.random.uniform(-0.05, 0.05, n_nodes),
    'lon': center_lon + np.random.uniform(-0.05, 0.05, n_nodes)
})

# Dates, hours, prediction horizons, pollutants
dates = pd.date_range('2025-03-01', periods=7)
hours = np.arange(24)
horizons = np.arange(1, 9)
pollutants = ['PM2.5', 'PM10', 'NO2', 'CO', 'CO2', 'CH4']

# Create random pollution data
multi_idx = pd.MultiIndex.from_product(
    [dates, hours, horizons, pollutants, nodes['node_id']],
    names=['date', 'hour', 'horizon', 'pollutant', 'node_id']
)
pollution_df = pd.DataFrame(index=multi_idx).reset_index()
pollution_df['value'] = np.random.uniform(10, 1000, len(pollution_df))


2025-03-23 09:49:03.014 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [3]:
pollution_df

,date,hour,horizon,pollutant,node_id,value
0,2025-03-01,0,1,PM2.5,Node 1,394.790517
1,2025-03-01,0,1,PM2.5,Node 2,278.635541
2,2025-03-01,0,1,PM2.5,Node 3,830.450134
3,2025-03-01,0,1,PM2.5,Node 4,363.185793
4,2025-03-01,0,1,PM2.5,Node 5,288.125165
...,...,...,...,...,...,...
241915,2025-03-07,23,8,CH4,Node 26,190.712635
241916,2025-03-07,23,8,CH4,Node 27,416.867018
241917,2025-03-07,23,8,CH4,Node 28,494.061738
241918,2025-03-07,23,8,CH4,Node 29,690.976226
